In [1]:
d = range(2 * 10 ** 8)

In [2]:
del d

In [3]:
e = range(1 * 10 ** 8)

In [4]:
del e

In [7]:
temp = []
for x in xrange(4 * 10 ** 8, 6 * 10 ** 8):
    temp.append(1)

In [8]:
del temp

In [9]:
whos

Variable   Type    Data/Info
----------------------------
x          int     599999999


In [ ]:
6617 -> 5089
1924 -> 394

In [ ]:
# it appears size of the list is the main thing.
# It appears integers are released because appending different integers
# doesn't cause RAM to change size
# however there still is a difference in RAM when creating list of just 1's 
# which are fully released when list deleased.
# That's confusing as list size is supposed to matter except when list created from 1's.
# and then numbers are released from RAM.

In [ ]:
# does that simply imply that RAM comes in blocks that are not allowed to be let go?
# if this is true, then does it matter what the object type is?

In [5]:
d = [1] * (4 * 10 ** 8)

In [6]:
del d # 3450 -> 393

In [1]:
d = [1] * (2 * 10 ** 9)

In [2]:
del d # 3450 -> 393

In [1]:
d = range(2 * 10 ** 8)

In [2]:
del d # 6619 => 5091 # decrease relatively small

In [1]:
d = [1] * (2 * 10 ** 8)

In [2]:
del d # 1922 -> 394

In [5]:
d = [1] * (2 * 10 ** 9)

In [6]:
del d # 15698 -> 410

In [ ]:
# have to use different numbers;
# it appears to hit RAM min bound, need to actually
# use different numbers to qualify

In [1]:
d = []
start, end = 0, 1
n = 2 * 10 ** 8 / 5


for y in range(5):
    d = []
    for x in xrange(2 * 10 ** 8 / 5):
        d.append(x)
    
    # change length of list
    # 1643 -> 1336

In [2]:
del d

In [1]:
d = range(2 * 10 ** 8 / 5) # 1639 -> 1332; looks like the actual numbers don't matter

In [2]:
del d

In [1]:
import numpy as np

n = 100000000

d = []
for _ in xrange(2 * 10 ** 8 / n):
    d.extend(np.random.choice(range(n), n))

In [2]:
del d

In [ ]:
100000: 6598 -> 409
10000000: 6836 -> 648
100000000: 8945 -> 2757

In [ ]:
d

In [8]:
del d, e

In [10]:
2757 + 5091 - 400

7448

In [3]:
whos

Interactive namespace is empty.


In [1]:
set(range(2 * 10 ** 8))
1

1

In [2]:
range(2 * 10 ** 8, 4 * 10 ** 8)
1

1

In [4]:
range(2 * 10 ** 8) + range(2 * 10 ** 8)
1 # 12859 -> 9800

1

In [5]:
del d

In [1]:
(range(4 * 10 ** 8)); # -> 9800

In [1]:
%time d = {i: 2 * i for i in xrange(1 * 10 ** 8)}
del d # 11256 -> 5101

CPU times: user 9.18 s, sys: 4.17 s, total: 13.3 s
Wall time: 13.3 s


In [ ]:
import numpy as np

n = 100000000

d = []
for _ in xrange(2 * 10 ** 8 / n):
    d.extend(np.random.choice(range(n), n))

In [1]:
# Python 2
skip = int(1e4)
x = []

for i in range(2 * 10 ** 8 / skip):
    # x.append(range(skip)) # 6512 -> 6512 -> 4168 # same with dict
    # x.append(range(i * skip, (i + 1) * skip)) # 6631 -> 6630 -> 6628
# appears that distinct numbers take relatively little space

In [3]:
whos

Variable   Type    Data/Info
----------------------------
i          int     19999
skip       int     10000


In [1]:
# Python 2
skip = int(1e1)
x = {}

for i in range(2 * 10 ** 8 / skip):
    # x[i] = range(skip) # 4877 -> 2711
    # x[i] = range(i * skip, (i + 1) * skip) # 9576 -> 7410
# not sure what's taking the big difference in RAM

In [2]:
del x

In [1]:
# Python 3
skip = int(1e4)
x = []

for i in range(int(2 * 10 ** 8 / skip)):
    # x.append(list(range(skip))) # 8105 -> 1324 -> 1325 # same with dict
    # x.append(list(range(i * skip, (i + 1) * skip))) # 8262 -> 1433 -> 1433
# appears that distinct numbers take relatively little space
# doesn't release all memory

In [ ]:
# Python 3
skip = int(1e1)
x = {}

for i in range(int(2 * 10 ** 8 / skip)):
    # x[i] = list(range(skip)) # 5643 -> 442 -> 443
    # x[i] = list(range(i * skip, (i + 1) * skip)) # 11858 -> 443 -> 445
# not sure what's taking the big difference in RAM
# releases all memory

In [ ]:
# RAM with cluster started -> RAM after processes calculated -> RAM after cluster killed
# 687 -> 1458 -> 714; 2:31; 5 times; purge and close
# 695 -> 1405 -> 745; 4:40; 5 times; purge, no close
# 684 -> 2624 -> 1951; 2:36; 5-times; no purge, do close [weird but correct result with RAM]
# 689 -> 1790 -> 1153; 4:55; 5-times; no purge and no close

# looks like purging saves lots of RAM, probably deletes main hub results but RAM still exists in each kernel
# looks like closing saves lots of time

# further research; take a look at if using same Client, does processing slow down and RAM go up?

In [ ]:
from ipyparallel import Client
from tqdm import tqdm
import time

def f(x):
    return 'whatever' + str(x)

for i in tqdm(range(5)):
    rc = Client()
    lv = rc.load_balanced_view()
    ar = lv.map_sync(f, range(10000))
    rc.purge_everything()
    rc.close()
    del ar
    del lv
    del rc

In [ ]:
!ipcluster stop

In [ ]:
!ipcluster start --daemonize

In [ ]:
from ipyparallel import Client
import os
import psutil
import time

def gen_large_arr(size):
    import numpy as np
    np.arange(size)
    return size, size * 2

rc = Client()
lv = rc.load_balanced_view()

arr_s = 0.
amr = lv.map_async(gen_large_arr, [3e8] * 16)
tic = time.time()
for i,(result, _child) in enumerate(zip(amr, amr._children)):
    print((i, psutil.Process(os.getpid()).memory_info().rss / 1e9, time.time() - tic))
#    arr_s = arr_s + result
#    _child._result = None

In [2]:
temp = []
for i in range(5):
    for x in xrange(2 * 10 ** 8 / 5):
        temp.append(x)
#    temp.pop()

In [5]:
del temp

In [4]:
whos

Variable   Type    Data/Info
----------------------------
temp       list    n=200000000
x          int     199999999


In [1]:
def foo(x):
    d = range(2 * 10 ** 8)
    return x

In [2]:
foo(42)

42

In [1]:
%time d = {x:x for x in range(2 * 10 ** 8)}

CPU times: user 15.1 s, sys: 7.15 s, total: 22.3 s
Wall time: 22.3 s


In [2]:
del d

In [12]:
while True:
    d = {x:x for x in range(2 * 10 ** 8)}
    del d

KeyboardInterrupt: 

In [13]:
del d

NameError: name 'd' is not defined

In [1]:
temp = range(5 * 10 ** 8)

In [1]:
import numpy as np

temp_list = []

my_dict = {}

In [2]:
from tqdm import tqdm

for i in tqdm(range(50)):
    my_dict[i] = list(np.random.uniform(10000000, size=int(1e7)))

100%|██████████| 50/50 [00:32<00:00,  1.54it/s]


In [3]:
del my_dict

In [ ]:
15866

In [1]:
import numpy as np
while True:
    %time d = {x: 2 * x for x in np.random.uniform(10000000, size=int(1e8))}

CPU times: user 1min 1s, sys: 6.45 s, total: 1min 8s
Wall time: 1min 4s
CPU times: user 1min 27s, sys: 5.23 s, total: 1min 32s
Wall time: 1min 31s


KeyboardInterrupt: 

In [2]:
whos

Variable   Type      Data/Info
------------------------------
d          dict      n=100000000
np         module    <module 'numpy' from '/ho<...>ages/numpy/__init__.pyc'>


In [3]:
%time del d # 9746

In [30]:
temp_list = range(int(3e8))

In [31]:
temp_list1 = range(int(3e8))

In [35]:
whos

Variable   Type      Data/Info
------------------------------
d          dict      n=99999999
gc         module    <module 'gc' (built-in)>
np         module    <module 'numpy' from '/ho<...>ages/numpy/__init__.pyc'>


In [10]:
temp_list = range(start, end)

In [11]:
del temp_list

In [7]:
import sys

start = 0
end = 5 * 10 ** 8
temp_list = []
while True:
    temp_list = range(start, end)
    del temp_list
    start, end = end, end + (end - start)
    print(start)
    sys.stdout.flush()

500000000
1000000000
1500000000


KeyboardInterrupt: 

In [12]:
whos

Variable   Type      Data/Info
------------------------------
end        int       2000000000
gc         module    <module 'gc' (built-in)>
start      int       1500000000
sys        module    <module 'sys' (built-in)>


In [4]:
import numpy as np


while True:
    temp1 = []
    temp2 = list(np.random.uniform(10000000, size=int(3e8)))
    temp1.extend(temp2)

KeyboardInterrupt: 

In [6]:
del (temp2)

In [3]:
whos

Interactive namespace is empty.


In [ ]:
324 -> 17333 -> 5023 -> 5023 # after gc

dictionary has residue; lists seem to have no residue;
actually it looks create large list and delete it leaves residue, which can be done on other kernels. None of the kernels contain any objects but they occupy the RAM. This implies that the garbage collector is correctly removing numbers but not releasing the full memory of the deleted object to the OS. Even if list generated and not returned in function call, RAM is not fully released.

In [ ]:
import ipyparallel as ipp

client = ipp.Client(profile='mycluster')
print(client.ids)
load_balanced_view = client.load_balanced_view()

In [ ]:
@load_balanced_view.parallel()
def silly_function(x):
    import numpy as np
    temp_list = []
    for i in range(3):
        temp_list.extend(np.random.uniform(10000000, size=int(1e7)))
    return x

In [ ]:
async = silly_function.map(range(4))